In [4]:
import requests
import re
import time
import os
import pandas as pd
import json
import sys
import warnings
warnings.filterwarnings(action='ignore')
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Hannanum

In [5]:
def get_page_sources_youtube(url):
    
    browser = webdriver.Chrome("./chromedriver.exe")
    page_sources = []
    
    # Move to the page
    url = url
    browser.get(url)
    
    # Scroll down
    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")
    
    # Iterate until end script of the website
    while True:
        
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3.0)
        
        new_page_height = browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        if new_page_height == last_page_height:

            break
        
        else:
            print("Continue")

        last_page_height = new_page_height

        print("End of scrolling")
        
        page_source = browser.page_source
        page_sources.append(page_source)  
    
    return page_sources

In [19]:
def get_info(url):

    url = url
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36 Edg/101.0.1210.39"}
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")   
    
    info = []
    
    video_name = soup.find('meta', itemprop="name")['content']
    num_views = soup.find('meta', itemprop="interactionCount")['content']
    date_published = soup.find('meta', itemprop="datePublished")['content']
    
    info.append([video_name, num_views, date_published])
    
    video_info = pd.DataFrame(index = ["Info"], columns = ['Video_name', 'Num_views', 'Date_published'], 
                              data = info).transpose()
    return video_info

def get_Comment_Like(url):
    
    browser = webdriver.Chrome("./chromedriver.exe")
    comment = []
    comment_date= []
    num_like = []

    # Move to the page
    url = url
    browser.get(url)
    
    # Scroll down and wait to get the reply
    
    browser.execute_script("window.scrollTo(0, 800);")
    time.sleep(10)
    browser.execute_script("window.scrollTo(800, 0);")
    time.sleep(5)
    browser.execute_script("window.scrollTo(10, document.documentElement.scrollHeight);")
    time.sleep(5)
    
    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")
    
    # Iterate until end script of the website
    while True:
        
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3.0)
        
        new_page_height = browser.execute_script("return document.documentElement.scrollHeight")
        
        if new_page_height == last_page_height:
            print("End scrolling")
            print("---"*30)
            print("Start Scrapping")
            break
        
        else:
            print("Iterating...")
            
        last_page_height = new_page_height
       
    # Get the comment
    for comments in browser.find_elements_by_id("content-text"):
        comment.append(comments.text)
        
    # Get year of the comment("~months,weeks,days" will be 2022 and others will be "2022 - ~years" in this code)
    for dates in browser.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a'):
        comment_date.append(dates.text)
        
    # Get the num of likes
    for likes in browser.find_elements_by_id("vote-count-middle"):

        if likes.text:
            num_like.append(likes.text)
        else:
            num_like.append("0")
            
            
            
    browser.quit()    
    # Make the dataframe with scrapped data
    video_content = pd.DataFrame({"Comment": comment, "Comment_date": comment_date, "Likes": num_like})
    video_content.Comment_date = video_content.Comment_date.apply(change_year)
    video_content.Likes = video_content.Likes.apply(convert_str_to_number)
    
    return video_content

def convert_str_to_number(x):
    
    total_stars = 0
    num_map = {'천': 1000, '만': 10000, 'K':1000, 'M':1000000, 'B':1000000000}
    
    if x.isdigit():
        total_stars = int(x)
    else:
        if len(x) > 1:
            total_stars = float(x[:-1]) * num_map.get(x[-1].upper(), 1)
            
    return int(total_stars)

def change_year(w):
    w = w.strip()
    w = re.sub(r"\(수정됨\)", " ", w)
    w = re.sub(r"전", " ", w)
    w = re.sub(r"1년", "2021", w) # Change 1 year ago to 2021
    w = re.sub(r"2년", "2020", w) # Change 2 year ago to 2020
    w = re.sub(r"3년", "2019", w) # Change 3 year ago to 2019
    w = w.strip()
    w = re.sub("([0-9]+)([^0-9]+$)", "2022", w) # Change any other period to 2022
    return w

In [17]:
url = "https://www.youtube.com/watch?v=zA0oeoXNM0c&ab_channel=ChrisLuno"
test_info = get_info(url)
test_df = get_Comment_Like(url)

Page0
Page0
Page0
Page0
Page0
Page0
Page0
Page0
Page0
Page0
Page0
Page0
End scrolling
------------------------------------------------------------------------------------------
Start Scrapping


In [20]:
test_info

,Info
Video_name,house & deep house vinyl mix
Num_views,547091
Date_published,2020-02-09


In [18]:
test_df

,Comment,Comment_date,Likes
0,i swear every vinyl dj goes and grabs every pl...,2021,941
1,"You know, i used to study while listening LoFi...",2022,3
2,"first of all, what a beautiful selection for n...",2021,160
3,Am I the only one who really enjoys when the ...,2021,52
4,Tracklist:\n00:00 Frits Wentink - Theme 8\n04:...,2020,121
...,...,...,...
268,PLAMP,2021,0
269,36:44 jajajaj,2021,1
270,some really poor mixing... wtf?,2021,0
271,FUCKIN ADVERTISING FROM YT,2022,0


In [48]:
pd.concat([test_info,test_df]).to_csv("Test.csv", encoding = "utf-8-sig", index = True)

In [17]:
df.to_csv("Test.csv", encoding = "utf-8-sig", index = False)